In this notebook the eQTLs and pQTLs indentified with statistical package MatrixEQTL are analysed with respect to the network of protein-protein interactions in yeast.

In [1]:
%matplotlib inline

# utilities
import gc
import time
import os

import collections
import itertools
from importlib import reload
import pickle

# data analysis tools
import joblib
import random
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests
from tqdm import *
import qgrid

# network analysis tools
import igraph as ig

# multiprocessing tools
import subprocess
import multiprocessing as mp

os.chdir("/home/vvi/Science/eQTL_analysis/")

# visualization tools
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import networks
import qtls
import util
import ontologies

gc.enable()

%autosave 15

Autosaving every 15 seconds


## Datasets used

Take a look at readme.md in /data folder for data description. (currently unavailable)

In [2]:
random.seed(int(time.time()))

# 112 segregants genotyped by inherited marker variants
full_genotypes_df = pd.read_table("./data/genotypes/processed_genotypes.csv")

''' Where possible, gene names were converted from systematic to standard notation '''
# mRNA expression and genotypes of strains the data is available for
eQTLs_expression_df = pd.read_table("./data/eQTLs/expression.csv")
eQTLs_genotypes_df = pd.read_table("./data/eQTLs/genotypes.csv")
# Protein expression and genotypes of strains the data is available for 
pQTLs_expression_df = pd.read_table("./data/pQTLs/expression.csv")
pQTLs_genotypes_df = pd.read_table("./data/pQTLs/genotypes.csv")

# pandas DataFrame —> numpy matrix = 2x speedup
# pQTLs_expression_mx = pQTLs_expression_df.as_matrix(columns=pQTLs_expression_df.columns[1:])
# pQTLs_genotypes_mx = pQTLs_genotypes_df.as_matrix(columns=pQTLs_genotypes_df.columns[1:])

# QTLs estimated with MatrixEQTL package for R
eQTLs_df = pd.read_table("./data/eQTLs/qtls.csv").query("q_value <= 0.05")
# eQTLs_df["gene"] = eQTLs_df["gene"].apply(networks.convert_to_std)
pQTLs_df = pd.read_table("./data/pQTLs/qtls.csv").query("q_value <= 0.05")

# Physical and genetic interactions taken from TheBioGrid project
full_interactome_df = pd.read_table("./data/interactions/yeast_interactome.csv")
# full_interactome_df = full_interactome_df[full_interactome_df['Publication year'] <= 2011]

genetic_interaction_categories = [
    "additive genetic interaction defined by inequality",
    "suppressive genetic interaction defined by inequality",
    "synthetic genetic interaction defined by inequality"
]
physical_interaction_categories = [
    "association",
    "colocalization",   
    "direct interaction",
    "physical association"
]

interaction_categories = genetic_interaction_categories + physical_interaction_categories
interactome_df = full_interactome_df.copy()
interactome_df.set_index('Interaction Type', inplace=True)
genetic_interactions_df = interactome_df[
    interactome_df.index.isin(genetic_interaction_categories)
]
physical_interactions_df = interactome_df[
    interactome_df.index.isin(physical_interaction_categories)
]

def extract_interactions(interactions_df):
    ''' Helper function to retrieve the edges of the graph of interactions '''
    return interactions_df[["Interactor A", "Interactor B"]].values

interactome_graphs_dict = {}
thebiogrid_modules_dict = collections.defaultdict(list)
for category_name in interaction_categories + ["all", "genetic", "physical"]:
    with open("./data/interactions/{}_interactions_graph.pkl".format(category_name), "rb") as pickle_file:
        interactome_graphs_dict[category_name] = pickle.load(pickle_file)
        thebiogrid_modules_dict[category_name] = interactome_graphs_dict[category_name].vs["name"]
interactome_graph = interactome_graphs_dict["physical"]

## Introduction and problem formulation

It seems rational to suppose that transcription and translation levels must be controlled by similar genetic mechanisms.  
Unfortunately, at the first sight there is no evidence to support that intuition: sets of eQTLs and pQTLs linked to given gene show no significant overlap for almost all genes. This is illustrated by the histogram below.

In [26]:
reload(util)
gene_pool = np.union1d(eQTLs_expression_df["gene"], pQTLs_expression_df["gene"])

linkage_overlap_data = []
for gene_name in gene_pool:
    linked_eQTLs = qtls.linked_markers(eQTLs_df, [gene_name])
    linked_pQTLs = qtls.linked_markers(pQTLs_df, [gene_name])
    linkage_overlap_data.append(qtls.jaccard(set(linked_eQTLs), set(linked_pQTLs)))

qtls.qtl_overlap_hist(linkage_overlap_data, "eQTLs_pQTLs")


Hypothesis arises: maybe not single genes, but functional modules tend to share linkages.  
The goal of this work is to test that hypothesis using yeast interactome data.

## Module processing

In [186]:
%%time

'''
    Randomize the interactome graph 1024 times and save each version. 
    These randomized copies will be used throughout the project to do perform PPI-network-based significance tests.
'''

def _produce_randomized_interactome_copy(i):
    randomized_interactome = interactome_graph.Degree_Sequence(
        interactome_graphs_dict["physical"].degree(),
        method='vl'
    )
    randomized_interactome.vs["name"] = interactome_graph.vs["name"]
    ig.write(
        graph=randomized_interactome,
        filename="./data/randomized_interactome_copies/{}.pkl".format(i),
        format="pickle"
    )

joblib.Parallel(n_jobs=mp.cpu_count())(
    joblib.delayed(_produce_randomized_interactome_copy)(i) 
    for i in range(1024)
)

CPU times: user 608 ms, sys: 244 ms, total: 852 ms
Wall time: 14min 40s


## Predicting pQTLs from eQTLs using PPI networks

Now, when approach using interaction networks is justified, it's time to provide solid ground: if significant fraction of pQTLs can be rediscovered from eQTLs linked to genes interacting with given (including itself), it will serve as an additional evidence for hypothesis being actually true (once again, no sophisticated clustering was performed here, while it would possibly yield better results).  

The main idea is not to test all (marker, gene) pairs but to filter for trusted only. In order to get rid of the pairs that will not link (or at least to dump more false than real linkages), the following approach is proposed: 
1. For each gene with measured protein abundance select its eQTL linkages as candidates into pQTLs. 
2. Extend eQTL linkages with those of the interacting genes. 

FDR decreases for the same q-value threshold, which results in more trustworthy observations.

Here is some statistics computed from some subgraphs of the interactome. 

**Physical interactions only**:   
- Common linkages: 2021, 32.7977929243752%
- Old linkages, total: 6162
- New linkages, total: 4048
- New linkages found: 2027

**Full interactome**:
- Common linkages: 3688, 59.85069782538137%
- Old linkages, total: 6162
- New linkages, total: 6219
- New linkages found: 2531

It becomes evident, that a significant fraction of estimated pQTLs can be reproduced using this approach.  
**Nevertheless, many questions arise**:
- Which fraction of trans-pQTLs is reproduced? (and is this approach capable of detecting those)
- Are all the hotspots reproduced? If some are not, why then? It is worth mentioning, that some of those were given cryptic names which are nowhere to be found. 
- How robust the results are? So far results are not compared with those computed from simulated, perturbed graphs. 

In [3]:
modules_type = "thebiogrid"
with open("./results/{}/modules_dict.pkl".format(modules_type), "rb") as pickle_file:
    modules_dict = pickle.load(pickle_file)
fdr_cutoffs = np.logspace(-5, -2, 10)

In [13]:
%%time

for simplify_flag in [False, True]:
    modules_stats = networks.basic_module_info(
         interactome_graph=interactome_graph, 
         modules_dict=modules_dict, 
         modules_type=modules_type,
         simplify=simplify_flag)
    modules_stats.to_csv("./results/{}/".format(modules_type) 
              + ("simplified" if simplify_flag else "raw") 
              + "_module_stats.csv", sep='\t', index=False)
    display(modules_stats.sort_values(by="genes_count", ascending=False))

,module_name,genes_count,genes_with_interactions,interactions_count
6,Mitosis & chromosome segregation,199,149,2478
12,"Glycosylation, protein folding|targeting, cell...",160,140,549
5,"Respiration, oxidative phosphorylation, mitoch...",138,135,318
15,Transcription & chromatin organization,110,85,1575
3,Cell polarity & morphogenesis,102,64,966
14,mRNA & tRNA processing,96,84,1126
4,Vesicle traffic,95,84,770
7,DNA replication & repair,95,75,926
0,rDNA & ncDNA processing,90,76,1055
13,Ribosome biogenesis,74,70,65


,module_name,genes_count,genes_with_interactions,interactions_count
6,Mitosis & chromosome segregation,199,145,667
12,"Glycosylation, protein folding|targeting, cell...",160,96,185
5,"Respiration, oxidative phosphorylation, mitoch...",138,88,165
15,Transcription & chromatin organization,110,81,303
3,Cell polarity & morphogenesis,102,60,225
14,mRNA & tRNA processing,96,80,456
4,Vesicle traffic,95,73,169
7,DNA replication & repair,95,71,183
0,rDNA & ncDNA processing,90,68,424
13,Ribosome biogenesis,74,35,45


CPU times: user 8.65 s, sys: 16 ms, total: 8.66 s
Wall time: 9.33 s


In [4]:
gene_pool = np.union1d(eQTLs_expression_df["gene"], pQTLs_expression_df["gene"])

In [56]:
jacc = []
for _ in range(1000):
    gene_set = np.random.RandomState().choice(gene_pool, 10, replace=False)
    linked_eQTLs = qtls.linked_markers(eQTLs_df, gene_set)
    linked_pQTLs = qtls.linked_markers(pQTLs_df, gene_set)
    jacc.append(qtls.jaccard(set(linked_eQTLs), set(linked_pQTLs)))
print(np.mean(jacc), np.median(jacc))

0.05317220280074159 0.0


In [5]:
%%time
print(modules_type)
display(
    pd.DataFrame(
        [
            (q_cutoff, *qtls.qtl_overlap_test(eQTLs_df.query("q_value < {}".format(q_cutoff)), 
                                              pQTLs_df.query("q_value < {}".format(q_cutoff)), 
                                              gene_pool, modules_dict))
            for q_cutoff in 10.**np.arange(-8, -1)
        ],
        columns = ["q_value_cutoff", "real_modules", "random_modules"],
        dtype=np.float32
    )
)

kegg_modules


,q_value_cutoff,real_modules,random_modules
0,1.000000e-08,0.229927,0.031213
1,1.000000e-07,0.239130,0.052010
2,1.000000e-06,0.221354,0.067761
3,1.000000e-05,0.168459,0.087292
4,1.000000e-04,0.173217,0.107421
5,1.000000e-03,0.186992,0.120255
6,1.000000e-02,0.191358,0.160969


CPU times: user 1min 54s, sys: 108 ms, total: 1min 54s
Wall time: 1min 54s


In [6]:
%%time

intersection_j, randomized_intersection_j = \
    qtls.qtl_overlap_by_module_test(eQTLs_df, pQTLs_df, gene_pool, modules_dict)

qtls.qtl_overlap_hist(intersection_j, modules_type)
qtls.qtl_overlap_hist(randomized_intersection_j, modules_type + "_randomized")

random samples generated:   0%|          | 0/108 [00:00<?, ?it/s]

random samples generated:   1%|          | 1/108 [00:00<00:31,  3.38it/s]

random samples generated:   2%|▏         | 2/108 [00:00<00:30,  3.53it/s]

random samples generated:   3%|▎         | 3/108 [00:00<00:29,  3.53it/s]

random samples generated:   4%|▎         | 4/108 [00:01<00:29,  3.49it/s]

...

random samples generated:  97%|█████████▋| 105/108 [00:35<00:01,  2.99it/s]

random samples generated:  98%|█████████▊| 106/108 [00:35<00:00,  2.98it/s]

random samples generated:  99%|█████████▉| 107/108 [00:35<00:00,  2.98it/s]

random samples generated: 100%|██████████| 108/108 [00:36<00:00,  2.97it/s]

CPU times: user 39.8 s, sys: 184 ms, total: 40 s
Wall time: 39.8 s


In [29]:
%%time
reload(qtls)

modules_for_ppi_test = qtls.filter_modules(
    modules_type=modules_type,
    expression_dfs=[eQTLs_expression_df, pQTLs_expression_df],
    qtl_dfs=[eQTLs_df, pQTLs_df],
    qtl_types=["eQTLs", "pQTLs"],
    interactome_graph=interactome_graph,
    raw_modules_dict=modules_dict,
    q_thresholds=fdr_cutoffs
)

with open("./results/{}/modules_for_ppi_test.pkl".format(modules_type), "wb") as pickle_file:
    pickle.dump(modules_for_ppi_test, pickle_file)


CPU times: user 320 ms, sys: 0 ns, total: 320 ms
Wall time: 380 ms


In [24]:
%%time
reload(qtls)
for expression_df, qtl_df, qtl_type in [(eQTLs_expression_df, eQTLs_df, "eQTLs"),
                                        (pQTLs_expression_df, pQTLs_df, "pQTLs")]:
    qtls.plot_analysis_results(
        expression_df=expression_df,
        interactions_type="physical",
        interactome_graph=interactome_graph,
        modules_dict=modules_dict,
        modules_type=modules_type,
        qtl_df=qtl_df,
        qtl_type=qtl_type
    )
    


subdirectories processed:   0%|          | 0/8 [00:00<?, ?it/s]

subdirectories processed: 100%|██████████| 8/8 [00:00<00:00, 10655.58it/s]


graphs and hists plotted: 0it [00:00, ?it/s]


vignettes plotted: 0it [00:00, ?it/s]


subdirectories processed:   0%|          | 0/8 [00:00<?, ?it/s]

subdirectories processed: 100%|██████████| 8/8 [00:00<00:00, 15040.09it/s]


graphs and hists plotted: 0it [00:00, ?it/s]


vignettes plotted: 0it [00:00, ?it/s]

CPU times: user 164 ms, sys: 28 ms, total: 192 ms
Wall time: 187 ms


In [64]:
%%time 

reload(qtls)
module_name = "physical"
module_genes = modules_dict[module_name]
predictions_dict = {}
# for module_name, module_genes in tqdm(modules_dict.items()):
module_graph = interactome_graphs_dict["all"].subgraph(
    np.intersect1d(interactome_graphs_dict["all"].vs["name"], module_genes)
)
if len(qtls.linked_markers(pQTLs_df, module_genes)) == 0\
        or len(qtls.linked_markers(eQTLs_df, module_genes)) == 0:
    predictions_dict[module_name] = "No linkages"
else:
    predictions_dict[module_name] = qtls.PqtlPredictor(
        eQTLs_df[eQTLs_df["gene"].isin(module_graph.vs["name"]) & (eQTLs_df["q_value"] < 0.001)], 
        pQTLs_df[pQTLs_df["gene"].isin(module_graph.vs["name"]) & (pQTLs_df["q_value"] < 0.001)],
        eQTLs_expression_df, eQTLs_genotypes_df,
        pQTLs_expression_df, pQTLs_genotypes_df,
        full_genotypes_df,
        module_name, module_graph
    ).predict()

CPU times: user 37.8 s, sys: 3.33 s, total: 41.2 s
Wall time: 3min 1s


In [65]:
for marker_name in predictions_dict.keys():
    print(marker_name)
    print(predictions_dict[marker_name], '\n')

physical
Common linkages:	1790, 44.01278583722646%
Old linkages, total:	4067
New linkages, total:	2533
New linkages found:	743 



## GeneOntology

In [5]:
from goatools import obo_parser

# building the GeneOntology DAG
obodag = obo_parser.GODag(
    "./data/geneontology/go-basic.obo",
    optional_attrs=["relationship"]
)
from Bio.UniProt.GOA import gafiterator

# how many of them are present in actual yeast genome?
yeast_gaf = './data/geneontology/goa_yeast.gaf'
goid2genes = collections.defaultdict(list)
with open(yeast_gaf) as handle:
    for rec in gafiterator(handle):
        go_term = obodag[rec["GO_ID"]]
        for par in go_term.parents:
            goid2genes[par.id].append(rec["DB_Object_Symbol"])
            
for key, val in goid2genes.items():
    goid2genes[key] = np.unique(val)

physically_interacting_genes = set(interactome_graphs_dict["physical"].vs["name"])
geneontology_modules_dict = {
    obodag[go_id].name.replace('/', '|') : genes 
    for go_id, genes in tqdm(goid2genes.items()) 
    if 10 <= len(interactome_graphs_dict["physical"].subgraph(
            set(genes) & physically_interacting_genes
        ).simplify().vs.select(_degree_gt=0)) <= 150
}

with open("./results/geneontology/modules_dict.pkl", "wb+") as pickle_file:
    pickle.dump(geneontology_modules_dict, pickle_file)
# with open("./results/geneontology/modules_dict.pkl", "rb+") as pickle_file:
#     geneontology_modules_dict = pickle.load(pickle_file)

./data/geneontology/go-basic.obo: fmt(1.2) rel(2018-07-02) 47,231 GO Terms; optional_attrs(relationship)


  0%|          | 0/4136 [00:00<?, ?it/s]

 17%|█▋        | 687/4136 [00:00<00:00, 6862.20it/s]

 29%|██▉       | 1218/4136 [00:00<00:00, 6084.30it/s]

 47%|████▋     | 1960/4136 [00:00<00:00, 6528.53it/s]

 65%|██████▍   | 2682/4136 [00:00<00:00, 6686.99it/s]

 78%|███████▊  | 3226/4136 [00:00<00:00, 6437.86it/s]

 95%|█████████▍| 3915/4136 [00:00<00:00, 6509.87it/s]

100%|██████████| 4136/4136 [00:00<00:00, 6431.32it/s]

## KEGG pathways

In [6]:
%%time 
kegg_pathways_dict = {("-".join(key.split('-')[:-1])):value for key, value in 
                     ontologies.query_KEGG(database="pathway", organism="sce").items()}

with open("./results/kegg_pathways/modules_dict.pkl", "wb+") as pickle_file:
    pickle.dump(kegg_pathways_dict, pickle_file)
# with open("./results/kegg_pathways/modules_dict.pkl", "rb+") as pickle_file:
#     kegg_pathways_dict = pickle.load(pickle_file)

modules retrieved:   0%|          | 0/116 [00:00<?, ?it/s]

modules retrieved:   1%|          | 1/116 [00:01<02:12,  1.15s/it]

modules retrieved:   2%|▏         | 2/116 [00:02<02:12,  1.16s/it]

modules retrieved:   3%|▎         | 3/116 [00:07<04:25,  2.35s/it]

modules retrieved:   3%|▎         | 4/116 [00:08<03:47,  2.03s/it]

...

modules retrieved:  97%|█████████▋| 113/116 [01:55<00:03,  1.02s/it]

modules retrieved:  98%|█████████▊| 114/116 [01:56<00:02,  1.02s/it]

modules retrieved:  99%|█████████▉| 115/116 [01:57<00:01,  1.02s/it]

modules retrieved: 100%|██████████| 116/116 [01:58<00:00,  1.02s/it]

CPU times: user 7.33 s, sys: 308 ms, total: 7.64 s
Wall time: 2min


## KEGG modules

In [7]:
%%time
reload(ontologies)

kegg_modules_dict = {key.replace('=>', 'to'):value for key, value in 
                     ontologies.query_KEGG(database="module", organism="sce").items()}

with open("./results/kegg_modules/modules_dict.pkl", "wb+") as pickle_file:
    pickle.dump(kegg_modules_dict, pickle_file)


modules retrieved:   0%|          | 0/108 [00:00<?, ?it/s]

modules retrieved:   1%|          | 1/108 [00:00<01:18,  1.36it/s]

modules retrieved:   2%|▏         | 2/108 [00:02<02:22,  1.35s/it]

modules retrieved:   3%|▎         | 3/108 [00:03<01:59,  1.14s/it]

modules retrieved:   4%|▎         | 4/108 [00:04<01:48,  1.05s/it]

...

modules retrieved:  97%|█████████▋| 105/108 [01:21<00:02,  1.28it/s]

modules retrieved:  98%|█████████▊| 106/108 [01:22<00:01,  1.29it/s]

modules retrieved:  99%|█████████▉| 107/108 [01:23<00:00,  1.29it/s]

modules retrieved: 100%|██████████| 108/108 [01:23<00:00,  1.29it/s]

CPU times: user 3.08 s, sys: 148 ms, total: 3.22 s
Wall time: 1min 25s


## Generalization: processing human data

## ModuleQTLs: concept introduction and rationale

In [150]:
has_linkages = collections.defaultdict(collections.Counter)
for qtl_type, qtl_df in [("eQTLs", eQTLs_df), ("pQTLs", pQTLs_df)]:
    for gene_name in tqdm(np.intersect1d(eQTLs_expression_df["gene"], pQTLs_expression_df["gene"])):
        has_linkages[qtl_type][len(qtls.linked_markers(qtl_df, [gene_name])) > 0] += 1
has_linkages

  0%|          | 0/1276 [00:00<?, ?it/s]

  9%|▊         | 111/1276 [00:00<00:01, 1108.37it/s]

 16%|█▌        | 202/1276 [00:00<00:01, 1000.99it/s]

 22%|██▏       | 278/1276 [00:00<00:01, 919.50it/s] 

 29%|██▉       | 370/1276 [00:00<00:00, 918.12it/s]

...

 68%|██████▊   | 863/1276 [00:00<00:00, 1721.52it/s]

 81%|████████  | 1032/1276 [00:00<00:00, 1715.32it/s]

 98%|█████████▊| 1247/1276 [00:00<00:00, 1777.10it/s]

100%|██████████| 1276/1276 [00:00<00:00, 1771.79it/s]

defaultdict(collections.Counter,
            {'eQTLs': Counter({False: 729, True: 547}),
             'pQTLs': Counter({False: 685, True: 591})})

In [18]:
df = pd.read_table("./df.csv")
np.min([df["real"].min(), df["simulated"].min()])

0.0

In [6]:
from itertools import tee

def parse_roman(s):
    numerals = {'M':1000, 'D':500, 'C':100, 'L':50, 'X':10, 'V':5, 'I':1}
    cur, prev = tee(numerals[c] for c in reversed(s.upper()))
    return next(cur) + sum((cur, -cur)[cur < prev] for cur, prev in zip(cur,prev))

old_gen_df = pd.read_table("./supplementary_materials/josh_formatted_genotypes.csv")[["RQTL_name", "name", "chromosome", "position", "sequence"]]
new_gen_df = pd.read_table("./data/kruglyak_2018/eQTLs.csv")[["gene", "pmarker", "chr", "markerPlotCoordinate"]]
new_gen_df["chr"] = new_gen_df["chr"].apply(lambda s: s.strip('chr')).apply(parse_roman)

In [6]:
old_chr2markpos = {n : old_gen_df.groupby(by="chromosome").get_group(n) for n in range(1, 17)}
new_chr2markpos = {n : new_gen_df.groupby(by="chr").get_group(n) for n in range(1, 17)}

In [7]:
for n in range(1, 17):
    # old_marker_segments = sorted([(int(markpos) - 500, int(markpos) + 500) for markpos, seq 
    #                               in old_chr2markpos[n][["position", "sequence"]].values])
    old_marker_positions = old_chr2markpos[n]["position"].values
    new_marker_positions = [int(mrkr.split(":")[1].split('_')[0]) for mrkr in new_chr2markpos[n]["pmarker"]]
    print("chr{}:\t {}".format(n, len(np.intersect1d(old_marker_positions, new_marker_positions))))
    # contains = collections.Counter()    
    # for markpos in new_marker_positions:
    #     for markseg in old_marker_segments:
    #         if markseg[0] <= markpos <= markseg[1]:
    #             contains[markseg] += 1
    # display(sum(contains.values()))

chr1:	 0
chr2:	 1
chr3:	 0
chr4:	 2
chr5:	 1
chr6:	 0
chr7:	 0
chr8:	 2
chr9:	 1
chr10:	 0
chr11:	 0
chr12:	 2
chr13:	 3
chr14:	 2
chr15:	 3
chr16:	 0


In [ ]:
count = collections.Counter()
by_marker = new_gen_df.groupby("pmarker")
for marker in new_gen_df["pmarker"]:
    count[marker] = len(by_marker.get_group(marker))
count.most_common(10)

In [14]:
namegroups = collections.Counter()
groups = old_gen_df.groupby("name") 
for name in set(old_gen_df["name"]): 
    namegroups[networks.convert_to_std(name)] = len(groups.get_group(name))
print(namegroups.most_common(15))

[('gKR08', 47), ('gOL01', 38), ('gKR07', 34), ('COS8', 23), ('ENA5', 16), ('YJL218W', 15), ('YER188W', 15), ('FLO10', 15), ('YKR104W', 14), ('PRM8', 14), ('COS6', 14), ('HO', 14), ('REE1', 14), ('NAR005C', 14), ('ARN2', 13)]


In [16]:
marker_count = 0
for marker_name in eQTLs_df["SNP"]:
    marker_count += namegroups[marker_name]
print(marker_count, eQTLs_df.shape[0])

48637 26540


In [70]:
print(pQTLs_df.query("q_value <= 0.001").shape)

(4231, 5)


In [31]:
%timeit qtls.linkage_similarity(interactome_graph, networks.graph_from_edges(eQTLs_df[["SNP", "gene"]].values))

399 ms ± 38.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
old_pqtls = pd.read_table("./data/pQTLs/qtls.csv")
# np.intersect1d(old_pqtls[["SNP", "gene"]].values, new_pqtls[["SNP", "gene"]].values)
new_pqtls = pd.read_table("./data/pQTLs/new_results.csv")
%timeit len(set(map(tuple, old_pqtls[["SNP", "gene"]].values)) & set(map(tuple, new_pqtls[["SNP", "gene"]].values))) / old_pqtls.shape[0]


28.5 ms ± 4.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
